# Optimization of sartorious transfer paramters of viscous liquids guided by Baysesian Optimization

## Summary

This notebook objective is to generate new suggestions of aspiration and dispense rates that will minimize the tansfer error while minimizing the time of transfer of a viscous liquid. The code in this notebook will aim to optimize the liquid transfer parameters  to minimize the mean error from three volumes (default 300, 500, 1000) and the time of transfer to dispense 1000 uL.
The code is strucutred as follows:
1.  Fisrt section is for importing the relevant packages to perform BO, inclduing Pytorch and BOtorch
2.  Second section includes the definition of the BO_LiqTransfer class that includes the method optimized_suggestions() that generates BO optimized aspiration and dispense rate values for a particular data set.
3. Third section includes the code to initialize the automated platform.
4. Fourth section includes the code to run experiments using M1 robotic arm attached with electronic pipette, automated mass balance and BO optimization of a viscous liquid. The steps for the optimziation are:

    i. Initilize a BO_LiTransfer objecet and load initilization data using data_from_csv() method

    ii. Run optimized_suggestions() method

    iii. Run liquid transfer gravimetric experiment using the best suggestion for aspiration and dispense rates

    iv. Update latest %error obtained from the transfer using suggested aspiration and dispense rates.
    
    v. Iterate through steps ii-iV

## 1. Imports

In [ ]:
# basic dependencies

import numpy as np
from numpy import loadtxt
from numpy import savetxt

import pandas as pd
import math
import time
from datetime import date

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

###########

# torch dependencies
import torch

tkwargs = {"dtype": torch.double, # set as double to minimize zero error for cholesky decomposition error
           "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu")} # set tensors to GPU, if multiple GPUs please set cuda:x properly

torch.set_printoptions(precision=3)

###########

# botorch dependencies
import botorch

# data related
from botorch.utils.sampling import draw_sobol_samples
from botorch.utils.transforms import unnormalize, normalize

# surrogate model specific
from botorch.models.gp_regression import SingleTaskGP
from botorch.models.model_list_gp_regression import ModelListGP
from botorch.models.transforms.outcome import Standardize
from gpytorch.mlls.sum_marginal_log_likelihood import SumMarginalLogLikelihood
from botorch import fit_gpytorch_model

# qNEHVI specific
from botorch.acquisition.multi_objective.objective import IdentityMCMultiOutputObjective
from botorch.acquisition.objective import IdentityMCObjective
from botorch.acquisition import qNoisyExpectedImprovement

# utilities
from botorch.optim.optimize import optimize_acqf
from botorch.sampling import SobolQMCNormalSampler
from botorch.utils.multi_objective.pareto import is_non_dominated
from botorch.utils.multi_objective.hypervolume import Hypervolume
from typing import Optional
from torch import Tensor
from botorch.exceptions import BadInitialCandidatesWarning

from gpytorch.constraints import GreaterThan
from torch.optim import SGD
from gpytorch.mlls import ExactMarginalLogLikelihood

import warnings

warnings.filterwarnings('ignore', category=BadInitialCandidatesWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)


# plotting dependencies
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline

SMALL_SIZE = 14
MEDIUM_SIZE = 18
BIGGER_SIZE = 20

plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title


## 2. BO_LiqTransfer class

In [ ]:
class BO_LiqTransfer:

    def __init__(self, liquid_name):
        self.liquid_name = liquid_name
        self._data = None
        self.features = ['aspiration_rate','dispense_rate']
        self.objectives = ['%error']
        self.bmax = 1.25
        self.bmin = 0.1
        self._latest_suggestion = None
        self._latest_volume = None
        self._latest_acq_value = None
        self.mean_volumes = [100,500,1000]
    
   
    def set_data(self,df):
        df['time_asp_1000'] = 1000/df['aspiration_rate'] + 1000/df['dispense_rate'] + df['delay_aspirate'] + df['delay_dispense']
        if 'acq_value' not in df.columns:
            df['acq_value'] = None

        if df.loc[:,self.features].duplicated().sum()==0:
            df_mean = df
        else:
            df_duplicates = df.where(df.duplicated(self.features,keep=False)==True).dropna(how='all')
            df_incomplete = df.where(df.duplicated(self.features,keep=False)==False).dropna(how='all')
            df_mean = pd.DataFrame(columns= df.columns)
            for index,values in df_duplicates.drop_duplicates(self.features).iterrows():
                if len(df_duplicates.loc[index:index+2]) == len(self.mean_volumes):
                    mean_error =df_duplicates.loc[index:index+2,'%error'].abs().mean()
                    df_duplicates.loc[index,'%error'] = -mean_error
                    df_duplicates.loc[index, 'volume'] ='mean'+str(self.mean_volumes)
                    df_mean = pd.concat([df_mean,df.loc[index:index+2],df_duplicates.loc[[index]]])
                else:
                    df_incomplete = pd.concat([df_incomplete,df_duplicates.loc[index:index+2]]).drop_duplicates()
            df_mean = pd.concat([df_mean,df_incomplete])
            df_mean = df_mean.reset_index(drop=True)    
        self._data = df_mean
      

    def data_from_csv(self,file_name):
        data = pd.read_csv(file_name)
        self.set_data(data)



    def update_data(self,df):
        self._latest_volume = df['volume'].iloc[-1]
        updated_data = pd.concat([self._data,df],ignore_index=True)
        self.set_data(updated_data)
        return self._data
                                

    def xy_split(self):
        df_train = self._data.where(self._data['volume']=='mean'+str(self.mean_volumes)).dropna(how='all')
        x_train = df_train[self.features]
        y_train = df_train[self.objectives]
        return x_train,y_train

    def set_bounds(self, x_train):
        return torch.vstack([x_train[0]*self.bmin, x_train[0]*self.bmax])



    def fit_surrogate(self):
        x_train, y_train = self.xy_split()
        x_train = torch.tensor(x_train.to_numpy(dtype=float), **tkwargs)
        y_train = torch.tensor(y_train.to_numpy(dtype=float), **tkwargs)
        y_train = -torch.absolute(y_train)
        

        problem_bounds = self.set_bounds(x_train)
       
        train_x_gp = normalize(x_train, problem_bounds)
        model1 = SingleTaskGP(train_x_gp, y_train, outcome_transform=Standardize(m=1))
        mll1 = ExactMarginalLogLikelihood(model1.likelihood, model1)

        fit_gpytorch_model(mll1)
    
        return model1, train_x_gp, y_train, problem_bounds
    
        
    def optimized_suggestions(self, random_state= 42):
        if random_state != None:
            torch.manual_seed(random_state) 
        standard_bounds = torch.zeros(2, len(self.features), **tkwargs)
        standard_bounds[1] = 1
        model1, train_x_gp, y_train, problem_bounds = self.fit_surrogate()
        acq_func1 = qNoisyExpectedImprovement(
        model=model1,
        X_baseline=train_x_gp, # feed total list of train_x for this current iteration
        sampler=SobolQMCNormalSampler(sample_shape=512),
        prune_baseline=True, cache_pending=True)  # options for improving qNEHVI, keep these on  

        sobol1 = draw_sobol_samples(bounds=standard_bounds,n=512, q=1).squeeze(1)
        sobol2 = draw_sobol_samples(bounds=standard_bounds,n=512, q=1).squeeze(1)
        sobol_all = torch.vstack([sobol1, sobol2])
            
        acq_value_list = []
        for i in range(0, sobol_all.shape[0]):
            with torch.no_grad():
                acq_value = acq_func1(sobol_all[i].unsqueeze(dim=0))
                acq_value_list.append(acq_value.item())
                
        # filter the best 12 QMC candidates first
        sorted_x = sobol_all.cpu().numpy()[np.argsort((acq_value_list))]
        qnei_x = torch.tensor(sorted_x[-12:], **tkwargs)  
        # unormalize our training inputs back to original problem bounds
        new_x =  unnormalize(qnei_x, bounds=problem_bounds)
        new_x = pd.DataFrame(new_x.numpy(),columns=['aspiration_rate','dispense_rate'])
        new_x['acq_value'] = sorted(acq_value_list, reverse=True)[:12]
        self._latest_suggestion = new_x[['aspiration_rate','dispense_rate']].iloc[0]
        self._latest_acq_value = new_x['acq_value'].iloc[0]
        return new_x





        
    
    


## 3. Initialization of automated plarform

In [ ]:
#Import relevant python packages
import pandas as pd
import time
from matplotlib import pyplot as plt
from pathlib import Path
import sys
REPOS = 'GitHub'
ROOT = str(Path().absolute()).split(REPOS)[0]
sys.path.append(f'{ROOT}{REPOS}')

from polylectric.configs.SynthesisB1 import SETUP, LAYOUT_FILE

from controllably import load_deck      # optional
load_deck(SETUP.setup, LAYOUT_FILE)     # optional

platform = SETUP
platform.mover.verbose = False


In [ ]:
#Initialization of variables for platfomr objects
pipette= platform.setup
deck = platform.setup.deck
balance = platform.balance
balance_deck = deck.slots['1']
source = deck.slots['2']
tip_rack = deck.slots['3']
bin = deck.slots['4']
pipette.mover.setSpeed(50)
print(balance_deck)
print(source)
print(tip_rack)
print(bin)


In [ ]:
#Check if balance is connected
balance.zero()
balance.toggleRecord(True)
time.sleep(5)
print(balance.buffer_df.iloc[-1])
balance.toggleRecord(False)

In [ ]:
#Stablish initial height of liquid on the source vial
pipette_name = 'rLine1000'
initial_liquid_level = 11.5

## 4. Guided BO optimization
i.   Create BO_LiqTransfer object and load data set.

Please set liquid name and volume to transfer according to the experiment.

In [ ]:
# Change according to experiment
std = '204'
liquid_name = 'Viscosity_std_' + std 
density = 0.8639
# Do not change
liq = BO_LiqTransfer(liquid_name)
folder = (r"C:\Users\admin\Documents\GitHub\viscosity_liquid_transfer_Pablo\Sartorious_experiments\Initialisation_Data\\")
liq.data_from_csv(folder+'Initialisation_'+std+'.csv')
liq._data

In [ ]:
# Change according to experiment
std = '1275'
liquid_name = 'Viscosity_std_' + std 
density = 0.8736
# Do not change
liq = BO_LiqTransfer(liquid_name)
import os
REPO = 'viscosity_liquid_transfer_Pablo'
folder = os.getcwd().split(REPO)[0]+REPO+r'\Sartorious_experiments\Experimental_Data\\Batch_optimization_data\BOTorch\SOBO\\' #folder that data is saved to
liq.data_from_csv(folder+liquid_name+'_SOBO_BOTorch_exp3_v1.csv')
liq._data


ii.   Run BO_LiqTransfer.optimized_suggestions() method to obtain optimized aspiration and dispense rates.



In [ ]:
liq.optimized_suggestions()


In [ ]:
liquids_dict = {
  liquid_name :{
        "rLine1000": {
            "aspiration_rate": liq._latest_suggestion['aspiration_rate'], 
            "dispense_rate": liq._latest_suggestion['dispense_rate'], 
            "blow_out" : False, 
            "delay_aspirate" : 10, 
            "delay_dispense" : 10, 
            "delay_blow_out" : 0, 
            },
    }

}

iii. Run liquid transfer using the best suggestion for aspiration and dispense rates in OT2 notebook.


In [ ]:
pipette.liquid.eject()

In [ ]:
pipette.mover.connect()

In [ ]:
# pipette.mover.home()
# pipette.mover.setHandedness(False)
pipette.mover.safeMoveTo(bin.wells['A1'].top)

In [ ]:
#define volume
volume = 1000
liquid_level = initial_liquid_level
#Check if new tip is required
pipette.mover.setHandedness(False)
pipette.mover.setSpeed(50)

if pipette.liquid.isTipOn()== False:
    pipette.attachTip()

#Transfer start
start = time.time()

#Aspirate step
pipette.mover.safeMoveTo(source.wells['A1'].from_bottom((0,0,liquid_level-5)))
pipette.liquid.aspirate(volume, speed=liquids_dict[liquid_name][pipette_name]['aspiration_rate'])
time.sleep(liquids_dict[liquid_name][pipette_name]['delay_aspirate'])

pipette.touchTip(source.wells['A1'])

#Dispense step
pipette.mover.safeMoveTo(balance_deck.wells['A1'].from_top((0,0,-5)))

balance.tare()
balance.clearCache()
balance.toggleRecord(True)
time.sleep(5)


pipette.liquid.dispense(volume, speed=liquids_dict[liquid_name][pipette_name]['dispense_rate'])

time.sleep(liquids_dict[liquid_name][pipette_name]['delay_dispense'])

#Blowout step
if liquids_dict[liquid_name][pipette_name]['blow_out'] == True:
    pipette.liquid.blowout(home=False)
    time.sleep(liquids_dict[liquid_name][pipette_name]['delay_blow_out'])

#Transfer termination
finish = time.time()
time_m = finish - start

pipette.mover.safeMoveTo(source.wells['A1'].top)
time.sleep(5)
balance.toggleRecord(False)
if liquids_dict[liquid_name][pipette_name]['blow_out'] == True:
    pipette.liquid.home()

#Do blowout
pipette.liquid.blowout(home=False)
time.sleep(5)
pipette.touchTip(source.wells['A1'])
pipette.liquid.home()
time.sleep(5)
pipette.liquid.blowout(home=False)
time.sleep(5)
pipette.touchTip(source.wells['A1'])
pipette.liquid.home()
time.sleep(5)
pipette.liquid.blowout(home=False)
time.sleep(5)
pipette.touchTip(source.wells['A1'])
pipette.liquid.home()


In [ ]:
fig,axs = plt.subplots()

axs.plot(balance.buffer_df['Time'],balance.buffer_df['Mass'])

axs.set_xlabel('Time')
axs.set_ylabel('Mass')

plt.show()

In [ ]:
#Record trasnfer values 

m = (balance.buffer_df.iloc[-10:,-1].mean()-balance.buffer_df.iloc[:10,-1].mean())/1000 #-1: mass column #-10: = last 10 rows #:10 = first 10 rows
error = (m-density*volume/1000)/(density/1000*volume)*100

#New dataframe
df = pd.DataFrame(columns = ['liquid', 'pipette', 'volume', 'aspiration_rate', 'dispense_rate','blow_out', 'delay_aspirate', 'delay_dispense', 'delay_blow_out', 'density', 'time', 'm', '%error', 'acq_value', 'Transfer_Observation', 'Comment'])
df = df.astype({'liquid':str,'pipette':str,'blow_out':bool,'Transfer_Observation':str,'Comment':str})
df = pd.concat([df,pd.DataFrame(liquids_dict[liquid_name][pipette_name],index=[0])],ignore_index=True)
df.iloc[-1,-6] = time_m
df.iloc[-1,2] = volume
df.iloc[-1, 0] = liquid_name
df.iloc[-1, 1] = pipette_name
df.iloc[-1,-7] = density
df.iloc[-1, -5] = m
df.iloc[-1,-4]= error
df.iloc[-1, -3] = liq._latest_acq_value

In [ ]:
#Update liquid level
#liquid_level = liquid_level - 1*volume/1000
liquid_level = liquid_level - 1.2*m/density

In [ ]:
#Observe error made
df.tail(20)

In [ ]:
#Assign category of observation of transfer such as Incomplete Dispense, Incomplete Aspiration, 
#Incomplete Aspiration and Dispense, Complete Transfer. 
#Comment if any unexpected exprimental mistakes or changes were performed that have to be taken into account.
#Incomplete Aspiration and Dispense, Complete Transfer. 
df.iloc[-1,-2]= 'Complete Transfer'
df.iloc[-1,-1]= 'Blowout before'

In [ ]:
#Check if new tip is required
pipette.mover.setSpeed(50)
pipette.mover.setHandedness(False)

if pipette.liquid.isTipOn()== False:
    pipette.attachTip()

#setup for loops
#TO BE CHANGED
iterations = 1
volumes_list = [500, 300]

#NOT TO BE CHANGED
liquid_level = initial_liquid_level
counter = 1

#while loop
while counter <= iterations:
    #getting botorch suggestions + implementing it in liquids_dict
    liq.optimized_suggestions()
    liquids_dict = {
    liquid_name :{
            "rLine1000": {
                "aspiration_rate": liq._latest_suggestion['aspiration_rate'], 
                "dispense_rate": liq._latest_suggestion['dispense_rate'], 
                "blow_out" : False, 
                "delay_aspirate" : 10, 
                "delay_dispense" : 10, 
                "delay_blow_out" : 0, 
                },
        }
    }
    #for loop
    for item in volumes_list:
        volume = item
        #liquid transfer
        #transfer start
        start = time.time() 

        #aspirate step
        pipette.mover.safeMoveTo(source.wells['A1'].from_bottom((0,0,liquid_level-5))) 
        pipette.liquid.aspirate(volume, speed=liquids_dict[liquid_name][pipette_name]['aspiration_rate'])
        time.sleep(liquids_dict[liquid_name][pipette_name]['delay_aspirate'])

        pipette.touchTip(source.wells['A1']) 

        #dispense step
        pipette.mover.safeMoveTo(balance_deck.wells['A1'].from_top((0,0,-5))) 
        balance.tare() 
        balance.clearCache() 
        balance.toggleRecord(True) 
        time.sleep(5)
        pipette.liquid.dispense(volume, speed=liquids_dict[liquid_name][pipette_name]['dispense_rate'])
        time.sleep(liquids_dict[liquid_name][pipette_name]['delay_dispense'])

        #blowout step
        if liquids_dict[liquid_name][pipette_name]['blow_out'] == True: 
            pipette.liquid.blowout(home=False)
            time.sleep(liquids_dict[liquid_name][pipette_name]['delay_blow_out']) 

        #transfer termination
        finish = time.time() 
        time_m = finish - start

        pipette.mover.safeMoveTo(source.wells['A1'].top) 
        time.sleep(5)
        balance.toggleRecord(False) 
        if liquids_dict[liquid_name][pipette_name]['blow_out'] == True:
            pipette.liquid.home() 

        #do blowout
        pipette.liquid.blowout(home=False) 
        time.sleep(5)
        pipette.touchTip(source.wells['A1'])
        pipette.liquid.home()
        time.sleep(5)
        pipette.liquid.blowout(home=False)
        time.sleep(5)
        pipette.touchTip(source.wells['A1'])
        pipette.liquid.home()
        time.sleep(5)
        pipette.liquid.blowout(home=False)
        time.sleep(5)
        pipette.touchTip(source.wells['A1'])
        pipette.liquid.home()

        #record transfer values 
        #calculating mass error functions
        m = (balance.buffer_df.iloc[-10:,-1].mean()-balance.buffer_df.iloc[:10,-1].mean())/1000 
        error = (m-density*volume/1000)/(density/1000*volume)*100

        #making new dataframe + filling it in
        df = pd.DataFrame(columns = ['liquid', 'pipette', 'volume', 'aspiration_rate', 'dispense_rate','blow_out', 'delay_aspirate', 'delay_dispense', 'delay_blow_out', 'density', 'time', 'm', '%error', 'acq_value'])
        df = df.astype({'liquid':str,'pipette':str,'blow_out':bool})
        df = pd.concat([df,pd.DataFrame(liquids_dict[liquid_name][pipette_name],index=[0])],ignore_index=True)
        df.iloc[-1,-4] = time_m
        df.iloc[-1,2] = volume
        df.iloc[-1, 0] = liquid_name
        df.iloc[-1, 1] = pipette_name
        df.iloc[-1,-5] = density
        df.iloc[-1, -3] = m
        df.iloc[-1,-2]= error
        df.iloc[-1, -1] = liq._latest_acq_value

        #change liquid levels
        liquid_level = liquid_level - 1.2*m/density   

        #printing checks
        print("LIQUID LEVEL: " +str(liquid_level) + "   LIQUID CHANGE: " +str(1.2*m/density) + "   ITERATION: " + str(counter) + ", " + str(volume))     

        #liquid level checks
        if (1.2*m/density > 1.2) or (1.2*m/density < 0):
            break
        if (liquid_level > initial_liquid_level) or (liquid_level < 6):
            break

        #update main dataframe
        liq.update_data(df)
        

    if (1.2*m/density > 1.2) or (1.2*m/density < 0):
        break
    if (liquid_level > initial_liquid_level) or (liquid_level < 6):
        break  
    counter += 1




iv. Update latest %error obtained from the transfer using suggested aspiration and dispense rates.

In [ ]:
liq.update_data(df) 


In [ ]:

liq._data 

v. Iterate trhough last two code cells.

In [ ]:
"C:\Users\admin\Documents\GitHub\viscosity_liquid_transfer_Pablo\Sartorious_experiments\Experimental_Data\Batch_optimization_data\BOTorch\SOBO"

In [ ]:
#save after each standard-experiment iteration
import os
REPO = 'viscosity_liquid_transfer_Pablo'
folder = os.getcwd().split(REPO)[0]+REPO+r'\Sartorious_experiments\Experimental_Data\\Batch_optimization_data\BOTorch\SOBO\\' #folder that data is saved to
liq._data.to_csv(folder+liquid_name+'_SOBO_BOTorch_exp3_v1.csv', index = False)

In [ ]:
pipette.ejectTipAt(bin.wells['A1'].top)
pipette.mover.home()

In [ ]:
        pipette.liquid.blowout(home=False) 
        time.sleep(5)
        pipette.touchTip(source.wells['A1'])
        pipette.liquid.home()
        time.sleep(5)
        pipette.liquid.blowout(home=False)
        time.sleep(5)
        pipette.touchTip(source.wells['A1'])
        pipette.liquid.home()
        time.sleep(5)
        pipette.liquid.blowout(home=False)
        time.sleep(5)
        pipette.touchTip(source.wells['A1'])
        pipette.liquid.home()

In [ ]:
pipette.liquid.eject()

In [ ]:
pipette.mover.connect()

In [ ]:
liq._data = liq._data.drop(35)

In [ ]:
pipette.liquid.dispense(1000)

In [ ]:
pipette.mover.connect()

In [ ]:
pipette.mover.home()

In [ ]:
pipette.liquid.connect()

In [ ]:
balance.connect()

In [ ]:
std = '1275'
liquid_name = 'Viscosity_std_' + std 
density = 0.8736
# Do not change
liq = BO_LiqTransfer(liquid_name)
folder = (r"C:\Users\admin\Documents\GitHub\viscosity_liquid_transfer_Pablo\Sartorious_experiments\Initialisation_Data")
liq.data_from_csv(liquid_name+'_BOTorch_exp3.csv')
liq._data